<a href="https://colab.research.google.com/github/gpicciuca/ai_engineering_master/blob/main/module1_python/Progetto_Python_Gestione_Rubrica_di_Contatti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Il ContactsManager gestisce interamente la rubrica attraverso un pandas DataFrame, salvato su file CSV ogni qualvolta si apportano modifiche ai dati,
e caricato automaticamente nonappena la classe viene istanziata.

Implementa diverse funzionalità, tra cui:
- Caricamento da file
- Salvataggio su file
- Aggiunta di un nuovo contatto
- Ricerca di contatti
- Modifica di contatti
- Eliminazione di contatti

In [ ]:
import pandas as pd
import os

class ContactsManager:

    """
    A simple contacts manager that helps you manage all your contacts
    with ease!
    """

    _DATABASE_NAME = "contacts.csv"
    # This list makes the dataset dynamic, allowing it to be extended easily
    _COLUMNS = ["Nome", "Cognome", "Cellulare", "Indirizzo"]

    def __init__(self):
        """
        Constructor method.
        Automatically loads the database from file, if it exists.
        Otherwise, creates a new, empty one.
        """
        self._df : pd.DataFrame = self.load_from_file(self._DATABASE_NAME)
        self._df["Cellulare"] = self._df["Cellulare"].astype(str)

    def __del__(self):
        """
        Destructor method.
        Automatically saves the current DataFrame to file.
        """
        self.save_to_file()

    def load_from_file(self, file_path):
        """
        Loads the specified file with pandas DataFrame.
        If loading succeeds, returns a DataFrame containing the loaded data.
        Otherwise an empty DataFrame with no records.
        """
        try:
            if os.path.exists(file_path):
                return pd.read_csv(file_path, index_col=0)
        except Exception as e:
            print(f"Si è verificato un errore durante il caricamento della rubrica {file_path}: {e}")

        return pd.DataFrame(columns=self._COLUMNS, index=[])

    def save_to_file(self):
        """
        Saves the current DataFrame to file.
        """
        self._df.to_csv(self._DATABASE_NAME, index=True)

    def get_columns(self):
        """
        Returns the column names of the DataFrame.
        """
        return self._COLUMNS

    def add_contact(self, *args):
        """
        Appends a new record to the DataFrame.
        *args must contain the exact number of elements matching the
        amount of columns available in the DataFrame!
        All fields are mandatory and cannot be left blank.
        Returns True if the record has been successfully added,
        False otherwise.
        """
        inputs = [*args]
        assert (len(inputs) == len(self._COLUMNS)), "Column count mismatch! Please check your input."

        if not all(map(lambda x : len(x) > 0, inputs)):
            return False

        if self._df.empty:
            self._df.loc[0, :] = inputs
        else:
            self._df.loc[self._df.index[-1] + 1, :] = inputs

        self.save_to_file()
        return True

    def show_contacts(self):
        """
        Prints all the contents of the DataFrame.
        Returns true if the DataFrame is not empty and data could be printed,
        false otherwise.
        """
        if not self.is_empty():
            print(self._df)
            return True
        return False

    def edit_contact(self, index, *args):
        """
        Updates an existing record in the DataFrame.
        *args must contain the exact number of elements matching the
        amount of columns available in the DataFrame!
        """
        inputs = [*args]
        assert (len(inputs) == len(self._COLUMNS)), "Column count mismatch! Please check your input."

        if not all(map(lambda x : len(x) > 0, inputs)):
            return False

        self._df.loc[index, :] = inputs
        self.save_to_file()
        return True

    def remove_contact(self, index):
        """
        Removes an existing record from the DataFrame.
        """
        self._df.drop(index, inplace=True)
        self.save_to_file()

    def find_contact(self, name = "", surname = ""):
        """
        Finds a contact within the DataFrame given the optionally supplied
        name and/or surname.
        Atleast one of the two must be non-empty.
        Returns a DataFrame containing all the matching records.
        """
        assert(len(name) > 0 or len(surname) > 0), "Please specify either a Name or a Surname!"

        if len(name) > 0 and len(surname) > 0:
            return self._df.loc[(self._df["Nome"] == name) & (self._df["Cognome"] == surname)]
        return self._df.loc[(self._df["Nome"] == name) | (self._df["Cognome"] == surname)]

    def is_empty(self):
        """
        Returns true if the DataFrame is empty, false otherwise.
        """
        return self._df.empty


Di seguito vi sono alcuni metodi di utilità, tra cui:
- La enum "Command", usata per gestire le varie opzioni del menu
- Il metodo "select_contact", che permette di cercare un contatto e, se ci sono piú di un match, ne permette la selezione, restituendo il contatto che si stava cercando
- Il metodo "prompt_contact_details", che semplifica la creazione dei prompt sulla base degli elementi definiti nel nostro ContactsManager
- Il metodo "prompt_edit_contact_details", che semplifica la creazione dei prompt per la modifica delle informazioni di un contatto selezionato

I 2 metodi di prompt permettono di rendere il programma più flessibile in quanto si ha la possibilità di aggiungere/rimuovere delle informazioni specifiche dal set dei dati che si vuole gestire.

In [ ]:
from enum import Enum

class Command(Enum):
    """
    Enumeration to improve readability in the state-machine within the main-loop.
    Defines all available actions the user can perform.
    """
    ADD_CONTACT = 1
    SHOW_CONTACTS = 2
    EDIT_CONTACT = 3
    REMOVE_CONTACT = 4
    FIND_CONTACT = 5
    EXIT = 6

def select_contact(manager: ContactsManager):
    """
    Searches for a specific contact by name and/or surname, and returns it.
    If the returned DataFrame contains more than one record, lets the user
    choose which one to select.
    Returns an empty DataFrame if no match has been found or if the user
    selected an invalid index.
    """
    name = input("Nome: ")
    surname = input("Cognome: ")

    contact = manager.find_contact(name, surname)

    if len(contact) >= 1:
        print("\n", f"Contatti trovati: {len(contact)}")
        print(contact)

    if len(contact) > 1:
        index = int(input("ID Utente da modificare: "))

        if index not in contact.index:
            print("ID Utente non valido!")
            return pd.DataFrame()

        # Select DataFrame row given by the index value specified by the user.
        # This will only get the Contact information whose index column
        # contains the exact value
        contact = contact.loc[[index]]

    return contact if len(contact) != 0 else pd.DataFrame()

def prompt_contact_details(manager: ContactsManager):
    """
    Dynamic prompt generation based on the columns defined in the ContactsManager.
    Returns a list containing the user inputs.
    """
    inputs = []
    for col in manager.get_columns():
        inputs.append(input(f"{col}: ").strip())
    return inputs

def prompt_edit_contact_details(manager: ContactsManager, contact: pd.DataFrame):
    """
    Dynamic prompt generation based on the columns defined in the ContactsManager.
    Used to edit an already selected contact, allowing the user to leave empty
    the fields they don't want to update.
    Returns a list containing the user inputs.
    """
    inputs = []
    for col in manager.get_columns():
        value = input("Nuovo %s [%s]: " % (col, contact[col].iloc[0])).strip()
        inputs.append(value if len(value) > 0 else contact[col].iloc[0])
    return inputs


In [ ]:
if __name__ == "__main__":
    manager = ContactsManager()

    while True:
      print("\nRubrica:")
      print(f"{Command.ADD_CONTACT.value}) Aggiungi un contatto")
      print(f"{Command.SHOW_CONTACTS.value}) Visualizza contatti")
      print(f"{Command.EDIT_CONTACT.value}) Modifica un contatto")
      print(f"{Command.REMOVE_CONTACT.value}) Elimina un contatto")
      print(f"{Command.FIND_CONTACT.value}) Cerca un contatto")
      print(f"{Command.EXIT.value}) Esci")
      print()

      cmd = input("Seleziona un'operazione [1-6]: ").strip()

      if not cmd.isnumeric():
          print("L'operazione scelta non è valida. Riprova...")
          continue

      print("\n")

      try:
          cmd = Command(int(cmd))

          if manager.is_empty() and cmd not in [Command.ADD_CONTACT, Command.EXIT]:
              print("La rubrica è vuota!")
              print("Aggiungi dei contatti prima di eseguire questa operazione.")
              continue

          if cmd == Command.ADD_CONTACT:
              print("Aggiungi un contatto")
              contact_details = prompt_contact_details(manager)
              if manager.add_contact(*contact_details):
                  print("Contatto aggiunto!")
              else:
                  print("Contatto non aggiunto! Si prega di ricontrollare i dati immessi.")
                  print("Tutti i campi sono obbligatori.")

          elif cmd == Command.SHOW_CONTACTS:
              print("Visualizza contatti")
              if not manager.show_contacts():
                  print("Non ci sono contatti salvati nella rubrica.")

          elif cmd == Command.EDIT_CONTACT:
              print("Trova contatto da modificare")
              contact = select_contact(manager)

              if contact.empty:
                  print("Nessun contatto trovato!")
                  continue

              print("\n", "Lasciare i campi vuoti per non apportare modifiche.")
              edited_contact_details = prompt_edit_contact_details(manager, contact)

              print("\n")
              confirm = input("Salvare le modifiche? [Y/N]: ").strip().upper()

              if confirm == "Y":
                  if manager.edit_contact(contact.index, *edited_contact_details):
                      print("Modifiche salvate!")
                  else:
                      print("Modifiche non apportate.")
                      print("Tutti i campi sono obbligatori e non possono essere")
                      print("lasciati vuoti!")
              else:
                  print("Modifiche non salvate!")

          elif cmd == Command.REMOVE_CONTACT:
              print("Trova contatto da eliminare")
              contact = select_contact(manager)

              if contact.empty:
                  print("Nessun contatto trovato!")
                  continue

              confirm = input("Eliminare il contatto? [Y/N]: ").strip().upper()

              if confirm == "Y":
                  manager.remove_contact(contact.index)
                  print("Contatto eliminato!")
              else:
                  print("Contatto non eliminato!")

          elif cmd == Command.FIND_CONTACT:
              print("Trova contatto")
              name = input("Nome: ")
              surname = input("Cognome: ")

              contact = manager.find_contact(name, surname)

              if len(contact) == 0:
                  print("Contatto non trovato!")
              else:
                  print("\n", contact)

          elif cmd == Command.EXIT:
              break

      except Exception as e:
          print(f"Si è verificato un'errore: {e}")

    # Make sure object is destroyed and destructor is called without relying
    # on the GC (Garbage Collector)
    del manager
